# Use Performance analyzer
The first tool we're looking at is the [Performance analyzer](https://learn.microsoft.com/en-us/power-bi/create-reports/desktop-performance-analyzer), which is built in to Power BI Desktop. The Performance analyzer helps you understand how report elements like visuals and DAX queries are performing.

The Performance analyzer helps you optimize at two of the four architecture levels, the **_data model_** and **_report visuals_**. The Performance analyzer is a great place to start when you're optimizing reports.

## Understand performance using Performance analyzer
The Performance analyzer displays and records logs that measure how each of your report elements performs when users interact with them. You can also see which aspects of their performance are most (or least) resource intensive. You can immediately see results, make changes, run the analyzer again, and see the impact of changes you've made.

The Performance Analyzer captures operations that occur in several major subsystems involved in executing a Power BI Report:
- **Report Canvas:** provides the user interface for Power BI reports including hosting visuals and filters, managing user interactions for consuming and authoring reports, and retrieving data for display. The Report Canvas is written using web technologies and runs in web browsers or web browser components. The Report Canvas retrieves data using a high-level, internal, Power BI query language known as Semantic Query.
- **Data Shape Engine (DSE):** evaluates Semantic Queries by generating and running one, or more DAX queries against a data model hosted inside Power BI, Power BI Desktop, Azure Analysis Services, or SQL Server Analysis Services.
- **Data Model Engine (AS):** stores the data model and provides services to reports, such as DAX query evaluation. The model may be hosted in Power BI, Power BI Desktop, Azure Analysis Services, or SQL Server Analysis Services. Depending on the data model host, a model may be tabular or multidimensional. Tabular models may contain in-memory tables, Direct Query tables, or a mix of such tables. DAX queries against tables in Direct Query mode will trigger queries to the Direct Query data source. For example, a DAX query against a Direct Query table backed by a SQL Server database will trigger one, or more, SQL queries.

## Use Performance analyzer
To use the Performance analyzer, enable the Performance analyzer on the View tab of the ribbon, and select **_Start recording_**.

<img src="../images/05_Work with semantic models in Microsoft Fabric/03/performance-analyzer-start.png" alt="Screenshot of view tab of the ribbon in Power BI desktop, with red box around Start recording button." style="border: 2px solid black; border-radius: 10px;">

After you start recording, take actions in the report. For example, move from one report tab to the next, select a slicer item, or interact with any of your visuals. Any actions you take in the report are displayed and logged in real time in the Performance Analyzer pane, in the order that the visual is loaded by Power BI.

The Performance analyzer is looking at the time it takes for each visual to query the data model and render results. This is the time from when a user does something on the page to when the visual is rendered, in milliseconds.

In the example below, three actions were taken after recording started.
1. Navigated to a new report page
2. Changed a slicer value
3. Cross highlighted a table

<img src="../images/05_Work with semantic models in Microsoft Fabric/03/performance-analyzer-results.png" alt="Screenshot of performance analyzer results after three report actions." style="border: 2px solid black; border-radius: 10px;">

Each visual's log information includes the time spent (duration) to complete the following categories of tasks:
- **DAX query** - if a DAX query was required, this is the time between the visual sending the query, and for Analysis Services to return the results.
- **Evaluated parameters** (preview) - time spent evaluating the field parameters within a visual. Learn more about [field parameters (preview)](https://learn.microsoft.com/en-us/power-bi/create-reports/power-bi-field-parameters).
- **Visual display** - time required for the visual to appear on the screen, including time required to retrieve any web images or geocoding.
- **Other** - time required by the visual for preparing queries, waiting for other visuals to complete, or performing other background processing.

After you've interacted with elements of the report you want to measure with Performance Analyzer, you can select the Stop button. The performance information remains in the pane after you select Stop for you to analyze.

You can select **_Refresh visuals_** in the Performance Analyzer pane to refresh all visuals on the current page of the report, which will gather information about all visuals.

If one particular visual appears to be performing slow, you can also refresh individual visuals. When Performance Analyzer is recording, you can select **_Analyze this visual_** found in the top-right corner of each visual, to refresh that visual, and capture its performance information.

<img src="../images/05_Work with semantic models in Microsoft Fabric/03/performance-analyzer-single-visual.png" alt="Screenshot of Analyze this visual icon on a Power BI table visual." style="border: 2px solid black; border-radius: 10px;">

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> As users interact with visuals in Power BI reports, DAX queries are submitted to the dataset and cached into memory. Because of this, you may need to clear the DAX query cache to get accurate results in the Performance analyzer. You can clear the cache by either closing and re-opening your report, or using DAX Studio.

## Evaluate performance data further
There are many different ways you can dig deeper into information recorded by the Performance analyzer.

You can use DAX Studio to investigate your queries in more detail by copying your query from the performance analyzer. After analyzing the query in DAX Studio, you can use your own knowledge and experience to identify where the performance issues are.

In the screenshot below, you can see that the DAX query in this table visual took 14 seconds. Users waited 14 seconds before seeing the results of the action that resulted in that query running, which in this case was a cross-highlight action. It's clear that this DAX needs to be optimized.

<img src="../images/05_Work with semantic models in Microsoft Fabric/03/performance-analyzer-copy-query.png" alt="Screenshot of Performance analyzer results, with a red box around the DAX query results." style="border: 2px solid black; border-radius: 10px;">

To analyze this query further, copy and paste the query into DAX Studio to repeat the execution of the query. In DAX Studio, you can activate more diagnostic tools using the Query Plan and Server Timings tracing options.

For DAX queries with long duration times, it's likely that a measure is written poorly or an issue has occurred with the data model. The issue might be caused by the relationships, columns, or metadata in your model, or it could be the status of the Auto date/time option.

You can also save Performance analyzer results by selecting the **_Export_** button, which creates a .json file containing results. Each event in the .json file contains timestamps, correlation information, and other metadata about the operation.

You can then load that data into DAX Studio to navigate through the performance metrics in more detail.

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> For more information about the contents of the .json export file, see the [Performance analyzer export file documentation](https://github.com/microsoft/powerbi-desktop-samples/commits/main/Performance%20Analyzer/Power%20BI%20Performance%20Analyzer%20Export%20File%20Format.docx).

# Troubleshoot DAX performance by using DAX Studio
In unit 2, we learned that Performance analyzer data can be further analyzed in DAX Studio. Import performance data into DAX Studio, where you can view, sort, and filter all performance data. You can also bring a single query into the editor where you can make adjustments to the query to improve performance.

## Understand the VertiPaq engine
Before jumping into optimization, it's important to understand what's happening under the hood of Power BI. By using compression algorithms and a multi-threaded query processor, the Analysis Services **_VertiPaq engine_** delivers fast access to tabular model objects and data by Power BI.

Power BI reads the content of your data source and transforms it in the internal VertiPaq columnar data structure, where each column is encoded and compressed. Dictionaries and indexes are created for each column. Finally, data structures are created for relationships and computation and compression of calculated columns occurs.

DAX queries are being processed by two engines, the **_formula engine_** and the **_storage engine_**. We'll talk more about these engines below.

## Describe DAX Studio
[**DAX Studio:**](https://daxstudio.org/) is an open-source tool for executing DAX queries against Power BI and Analysis Services models. DAX Studio is useful for:
- Reviewing the contents of your data model.
- Writing and optimizing complex DAX formulas and queries.

Download and install DAX Studio with the default settings, connect to your data model, and begin working on your queries. Once DAX Studio is installed, it can also be launched from the **External Tools** tab of the ribbon in Power BI desktop.

## Optimize the data model
Now that you have an idea of how the VertiPaq engine works, let's discuss how you can use DAX Studio to optimize DAX queries that run in this environment.

### Optimize DAX queries
Calculations using DAX, either measures or columns, are a part of a DAX query, which is processed by two engines in VertiPaq. When a query is processed, the **_formula engine_** processes the request, asks the storage engine for data, and performs necessary calculations. The **_storage engine_** retrieves and aggregates data requested by the formula engine.

In the diagram below, the DAX query is sent to the tabular model in steps 1 and 2. The request is then processed by the formula engine and sent to the storage engine, represented by step 3. In step 4, the storage engine either retrieves the data from the model and stores it in memory (for import mode), or passes the query on to the data source (for DirectQuery). For import mode, refreshing the data will retrieve the data from the source.

<img src="../images/05_Work with semantic models in Microsoft Fabric/03/vertipaq-engine-process.png" alt="Diagram of the VertiPaq engine process, beginning with a query that gets sent to the data source." style="border: 2px solid black; border-radius: 10px;">

Troubleshooting in DAX Studio enables you to see detailed statistics on the server timings of your query. You're able to view the proportion of time the query takes in each engine, and can then adjust your queries accordingly to improve performance.

Let's talk through a scenario to understand how you can optimize a query using DAX Studio.

You have a report that contains a matrix visualizing 6 measures. Your CEO informs you that the visual is slow to render, and therefore the report is unusable. You start to dig in and confirmed slow render times using the Performance analyzer in Power BI desktop.

You then copy the query to look at it in DAX Studio to get more information on what might be causing your problem. In DAX Studio, you clear the cache(1), turn on the server timings (2), and then run the query (3).

<img src="../images/05_Work with semantic models in Microsoft Fabric/03/dax-studio-process.png" alt="Diagram of the steps in DAX studio as described in the paragraph above." style="border: 2px solid black; border-radius: 10px;">

From top left to bottom right, the statistics tell you how many milliseconds it took to run the query, and the duration the storage engine (SE) CPU took. In this case, the formula engine (FE) took 73.5% of the time, while the SE took the remaining 26.5% of the time. There were 34 individual SE queries and 21 cache hits.

<img src="../images/05_Work with semantic models in Microsoft Fabric/03/server-timings.png" alt="Screenshot of server timings in DAX Studio." style="border: 2px solid black; border-radius: 10px;">

From here, you can investigate what in your measures might be causing the issues. This requires deep DAX knowledge and is sometimes a case of trial and error.

You experiment with the measures and recognize that improving the sales measure by replacing complicated `IF` statements in DAX with variables and a time intelligence function. A safe way to experiment is to comment out measures and rework them. You can comment out measures by typing two forward slashes at the beginning of a line (`//`). For multi-row comments, use `/*` at the beginning of the comment and `*/` to close the comment.

After experimenting, you clear the cache and run the query again with the updates you made to the measure. You find that your updated measure performs much better, with nearly a 50% reduction in query execution time.

<img src="../images/05_Work with semantic models in Microsoft Fabric/03/server-timings-reduced.png" alt="Screenshot of improved server timings in DAX Studio." style="border: 2px solid black; border-radius: 10px;">

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> The storage engine caches the results in memory for reuse. Because of this cache, it is critical to **clear the cache** prior to running queries in DAX Studio.

### View model metrics using VertiPaq Analyzer
Viewing the **VertiPaq Analyzer Metrics** in DAX Studio is a great way to get an overall view of what's going on in your data model. VertiPaq Analyzer reports the memory consumption of the data model and can be used to quickly identify where you're spending the most memory. In short - you can use VertiPaq Analyzer to make memory gobbling offenders obvious, rectify them in Power BI, and then rerun VertiPaq Analyzer to see the immediate benefits of your data model updates.

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> The VertiPaq engine only stores data in memory in import models. If you're using **_DirectQuery_**, the VertiPaq engine simply sends that query to the source. This means that viewing the VertiPaq Analyzer Metrics will only be helpful for **_import_** or **_composite_** models.

You can look at the size of the table, columns, etc., in bytes. The .pbix file further compresses these sizes - the displayed sizes in bytes are evaluated prior to compression.

To view model metrics, launch DAX Studio from the external tools tab of the Power BI ribbon and select View Metrics from the Advanced tab in DAX Studio.

<img src="../images/05_Work with semantic models in Microsoft Fabric/03/view-model-metrics.png" alt="Screenshot of Advanced tab in DAX Studio with cursor over View Metrics button." style="border: 2px solid black; border-radius: 10px;">

VertiPaq analyzer displays a number of important metrics about your model. We're going to focus specifically on memory consumption and cardinality. For a complete list of what each of these columns mean, consult the [DAX Studio documentation](https://daxstudio.org/docs/intro/).

Viewing metrics in DAX Studio helps you immediately find and fix problems. In this case, you can see that the problem is a column with high cardinality. You can then fix that issue back in Power BI, refresh the metrics, and immediately see the effects of your changes on the model.

For example, notice that the model in the image below contains a table that consumes 99.6% of the database memory. By drilling into the table, you can see that two columns, End date and Start date are gobbling up the most memory.

<img src="../images/05_Work with semantic models in Microsoft Fabric/03/vertipaq-analyzer-scan-results.png" alt="Screenshot of VertiPaq Analyzer scan results, showing that the trips table is consuming 99.6% of the database memory." style="border: 2px solid black; border-radius: 10px;">

Take a look at those two columns back in Power BI desktop and notice that they're Date/time columns. Date/time columns inherently have high cardinality due to all of the possible combinations of dates and times.

<img src="../images/05_Work with semantic models in Microsoft Fabric/03/power-bi-date-time-columns.png" alt="Screenshot of End date and Start date columns in Power BI, with a red box around the Date/time format." style="border: 2px solid black; border-radius: 10px;">

Using the VertiPaq Analyzer in DAX Studio can help you easily identify and eliminate columns with high cardinality (including auto Date/time and floating-point decimal data types), and identify and remove columns that aren't used for anything.

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> Refer to the [Power BI optimization guide](https://learn.microsoft.com/en-us/power-bi/guidance/power-bi-optimization#optimizing-the-data-model) for more detailed information on optimizing the data model.

# Optimize a data model by using Best Practice Analyzer
How can you be sure to implement data modeling best practices as you design and build data models in Power BI? There's a tool for that! The Best Practice Analyzer (BPA) in Tabular Editor can be used during the development of tabular models in Power BI or Analysis Services models.

## Describe Tabular Editor
Tabular Editor is an alternative tool for authoring tabular models for Analysis Services and Power BI. Tabular Editor 2 is an open-source project that can edit a BIM file without accessing any data in the model. Tabular editor enables users to easily manipulate tabular models and can be used in a few different ways. Tabular Editor can be used to:
- quickly edit data models
- automate repetitive tasks using scripting
- incorporate DevOps with tabular models
- run BPA rules

This unit focuses on using Tabular Editor to run the BPA to ensure you're implementing data modeling best practices as you build.

## Describe the Best Practice Analyzer (BPA)
BPA is a set of rules run in Tabular Editor that notify you of potential modeling missteps or changes that you can make to improve your model design and performance. It includes recommendations for naming, user experience, and common optimizations that you can apply to improve performance. For more information, see [Best practice rules to improve your model's performance](https://powerbi.microsoft.com/blog/best-practice-rules-to-improve-your-models-performance/).

BPA rules are a set of rules that you can add to your instance of Tabular Editor. When BPA rules are run on your tabular model, you'll get a list of rules that your model violates, and can fix them using Tabular Editor.

BPA includes a [set of pre-defined rules](https://powerbi.microsoft.com/blog/best-practice-rules-to-improve-your-models-performance/), and you can also define your own rules to encourage certain conventions and practices when developing tabular models.

To use the pre-defined BPA rules, download the [BPA rules .json file](https://github.com/microsoft/Analysis-Services/tree/master/BestPracticeRules) from the GitHub repository. The rules are divided into categories for easier viewing, for example:
- Performance
- DAX Expressions
- Error Prevention
- Formatting
- Maintenance
Each rule has a description, and many of the rules also have a reference article/video. Reading the rule description and article will provide context as to why the rule is important and why you should follow it.

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> BPA rules can be run against tabular models in Power BI Desktop, SQL Server Analysis Services, Azure Analysis Services, or Power BI Premium.

## Run BPA in Tabular Editor
1. Install [Tabular Editor](https://github.com/TabularEditor/TabularEditor).  
    _Tabular Editor has an open-source and a paid version. This unit refers to use of the open-source version only._
2. Download the Best Practice Rules from [GitHub](https://github.com/TabularEditor/BestPracticeRules/releases).
3. Save the file in `C:\Users\username\AppData\Local\TabularEditor` and name it **`BPARules.json`**.
4. In Power BI Desktop, select Tabular Editor on the External Tools tab of the ribbon.

<img src="../images/05_Work with semantic models in Microsoft Fabric/03/power-bi-external-tools.png" alt="Screenshot of the external tools tab of the ribbon in Power BI Desktop displaying Tabular Editor." style="border: 2px solid black; border-radius: 10px;">

    _Tabular Editor will automatically connect to the data model_
5. On the Tools tab, select **_Best Practice Analyzer_**.

<img src="../images/05_Work with semantic models in Microsoft Fabric/03/launch-best-practice-analyzer.png" alt="Screenshot of Tabular Editor interface with Best Practice Analyzer selected on the Tools tab." style="border: 2px solid black; border-radius: 10px;">

6. The Best Practice Analyzer window will open, displaying any violations of the Best Practice Rules.

<img src="../images/05_Work with semantic models in Microsoft Fabric/03/best-practice-analyzer-results.png" alt="Screenshot of Best Practice Analyzer window displaying 20 objects in violation of 7 best practice rules." style="border: 2px solid black; border-radius: 10px;">

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> The best practice analyzer scans your model for issues whenever a change is made to the model. This is on by default but can be disabled.

7. Review the objects in violation of the rules and fix them using Tabular Editor. For example, let's fix the columns violating the **Don't summarize numeric columns** rule, starting with the Weight column in the Product table.
8. Double select (or right select) the object to go to the object in Tabular Editor. Change the object properties as necessary. In this case, we're changing **Summarize by** from **Sum** to **None**.

<img src="../images/05_Work with semantic models in Microsoft Fabric/03/best-practice-analyzer-fix-object.png" alt="Screenshot of Tabular Editor displaying the object properties, with a blue highlight over Summarize by None." style="border: 2px solid black; border-radius: 10px;">

9. To save your changes to back to the Model.bim file, select **Save**, or use the keyboard shortcut **ctrl + s**.

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> Tabular Editor uses the [Tabular Object Model (TOM)](https://learn.microsoft.com/en-us/analysis-services/tom/introduction-to-the-tabular-object-model-tom-in-analysis-services-amo) to load and save metadata to and from Model.bim files. When fixing Best Practice rule violations, saving changes in Tabular Editor pushes modifications to the connected Power BI desktop data model.

Customize BPA for your organization
BPA was designed to enable you to create custom rules and best practices for your organization. If the Best Practice Rules don't suit your needs, you can create rules required for data modeling best practices as you deem appropriate.

You can also edit existing rules, disable and ignore rules, and set severity levels for each rule. From the Best Practice Analyzer window, select **Manage Best Practice Rules**. This will enable you to turn rules to use in your scan on and off, edit rules, and delete rules. Edits you make to Best Practice Rules in this window will then be saved to the .json file.

<img src="../images/05_Work with semantic models in Microsoft Fabric/03/best-practice-analyzer-manage-rules.png" alt="Screenshot of the Manage Best Practice Rules window where users can create, edit, and delete rules." style="border: 2px solid black; border-radius: 10px;">

### Incorporate the use of BPA into your Continuous Integration/Continuous Deployment (CI/CD) process
If your organization has an established CI/CD process using Git, BPA can be integrated into your Azure Pipelines. If objects violate Best Practice Rules in the build, you can establish a process to either fail the build or proceed with warning based on the severity of the violation.

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> The severity level that you set for each rule while managing your best practice rules only comes into play when deploying models using the [command line option](https://docs.tabulareditor.com/te2/Command-line-Options.html) in Tabular Editor.